# Tutorial 4: Other Functionalities
In this tutorial we show how different parts of the API work. First we import the need.

In [ ]:
import spyker
import torch
import numpy as np

We start this tutorials by Spyker tensors and how to convert them to and back from PyTorch tensors and Numpy arrays. Spyker tensors only support 32-bit floating point and 8-bit unsigned integer numbers on CPUs and CUDA GPUs.

In [ ]:
A = torch.rand(10, 10, dtype=torch.float32) # random pytorch tensor
B = np.random.random([10, 10]).astype(np.float32) # random numpy array
C = spyker.to_tensor(A) # convert tensor A into a spyker tensor
D = spyker.to_tensor(B) # convert array B into spyker tensor
E = spyker.to_torch(C) # convert spyker tensor to numpy array
F = spyker.to_numpy(D) # convert spyker tensor to pytorch tensor
print(torch.allclose(A, E))
print(np.allclose(B, F))

Spyker also has a `Sparse` container that can only contain 5D binary sparse values.

In [ ]:
A = (torch.rand(1, 1, 1, 10, 10) > .95).to(torch.uint8) # random 5D values
B = spyker.to_sparse(A) # convert to sparse
print(B.sparsity()) # print the sparsity of the data
C = spyker.to_torch(B.dense())
print(A.allclose(C))

Some of the modules of Spyker need to generate random data. You can set the seed of this random generation by `spyker.random_seed` function which takes in an integer seed. Some of the operations that Spyker performs need additional storage that needs to ba cached. These cached data don't get removed automatically and stay until the end of program. If you need to clear all of these cached data, you can use `spyker.clear_context` function. There is also a function that forces cuDNN to allocate less memory for the convolution operations which can be set with `spyker.light_conv`which takes in a boolean value setting it to true or false.

There are some information that you can get when you want to use the CUDA device. First, we want to know if cuda is available (your setup is correct and your build of Spyker supports your hardware).

In [ ]:
print('Is CUDA available?', spyker.cuda_available())
print('Count of CUDA devices you have:', spyker.cuda_device_count())
print('Supported CUDA architectures of your Spyker build:', spyker.cuda_arch_list())
print('Architectures of your cuda devices', spyker.cuda_device_arch())
print('Total memory avialable of current CUDA device:', spyker.cuda_memory_total())
print('free memory avialable of current CUDA device:', spyker.cuda_memory_free())

You can change the CUDA device that you are using by passing the index of your device `spyker.cuda_set_device` function. An important optimization that Spyker uses is caching cuda memory. This is enabled by default and can be changed using `spyker.cuda_cache_enable` which takes in a boolean value. We can see how this cache works.

In [ ]:
print('GPU memory Taken: ', spyker.cuda_memory_taken())
print('GPU memory Used', spyker.cuda_memory_used())
A = spyker.create_tensor(spyker.device('cuda'), 'f32', [10, 10]) # empty tensor
print('GPU memory Taken: ', spyker.cuda_memory_taken())
print('GPU memory Used', spyker.cuda_memory_used())
del A
print('GPU memory Taken: ', spyker.cuda_memory_taken())
print('GPU memory Used', spyker.cuda_memory_used())
spyker.cuda_cache_clear()
print('GPU memory Taken: ', spyker.cuda_memory_taken())
print('GPU memory Used', spyker.cuda_memory_used())

As we can see when we clear the cuda cache, the memory used drops back to zero, but the memory taken doesn't. We can also see a summary of the cache.

In [ ]:
A = spyker.create_tensor(spyker.device('cuda'), 'f32', [10, 10])
B = spyker.create_tensor(spyker.device('cuda'), 'f32', [5, 5])
C = spyker.create_tensor(spyker.device('cuda'), 'f32', [2, 2])
spyker.cuda_cache_print()